# Looks at correlations between dataset characteristics and percent error for artificial programs
Splits all ceo into subsets that serve as mock programs and then splits those further to calculate correlations 
between dataset characteristics and percent error for expected. 
The correlations found using each mock program as a starting point were different.

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from uuid import UUID

import matplotlib.pyplot as plt

import sys
sys.path.append('/Users/mallen2/alternate_branches/eval-compatible-server/e-mission-server')

import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq
import emission.core.wrapper.user as ecwu

import confusion_matrix_handling as cm_handling
from confusion_matrix_handling import MODE_MAPPING_DICT
import get_EC
import helper_functions as hf

import sklearn.model_selection as skm

from sklearn.model_selection import KFold
from sklearn import linear_model

import scipy
from itertools import chain

METERS_TO_MILES = 0.000621371 # 1 meter = 0.000621371 miles
ECAR_PROPORTION = 0 #0.01 #~1% of cars on the road are electric.
DROVE_ALONE_TO_SHARED_RIDE_RATIO = 1

df_EI = pd.read_csv(r'Public_Dashboard/auxiliary_files/energy_intensity.csv') # r stands for raw string, only matters if the path is on Windows

In [ ]:
%store -r expanded_labeled_trips

In [ ]:
# Alternatively you could run "Store_expanded_labeled_trips.ipynb" first and then this line.
# %store -r expanded_labeled_trips 

'''import database_related_functions as drf  # all the emission server functions for this notebook are in here.

user_list, os_map, uuid_program_map = drf.get_participants_programs_and_operating_systems()

# Takes ~ 1 min 45 s to 2 min 45 s on Macbook Pro for all ceo data up to May 2022.
expanded_labeled_trips = drf.get_expanded_labeled_trips(user_list)
expanded_labeled_trips['os'] = expanded_labeled_trips.user_id.map(os_map)
expanded_labeled_trips['program'] = expanded_labeled_trips['user_id'].map(uuid_program_map)

# Drop columns you don't need
expanded_labeled_trips = expanded_labeled_trips.drop(labels = ['source', 'end_fmt_time', 'end_loc', 'raw_trip',
    'start_fmt_time', 'start_loc','start_local_dt_year', 'start_local_dt_month', 'start_local_dt_day',
    'start_local_dt_hour', 'start_local_dt_minute', 'start_local_dt_second',
    'start_local_dt_weekday', 'start_local_dt_timezone',
    'end_local_dt_year', 'end_local_dt_month', 'end_local_dt_day',
    'end_local_dt_hour', 'end_local_dt_minute', 'end_local_dt_second',
    'end_local_dt_weekday', 'end_local_dt_timezone'], axis = 1)

expanded_labeled_trips['distance_miles'] = expanded_labeled_trips.distance*METERS_TO_MILES'''

In [ ]:
unit_dist_MCS_df = pd.read_csv("unit_distance_MCS.csv").set_index("moment")
energy_dict = cm_handling.get_energy_dict(df_EI)

In [ ]:
# Drop trips you want to exclude from analysis.
expanded_labeled_trips = hf.drop_unwanted_trips(expanded_labeled_trips,drop_not_a_trip=False)

# Find the primary mode - the sensed mode with the longest section for each trip.
expanded_labeled_trips = hf.get_primary_modes(expanded_labeled_trips,energy_dict,MODE_MAPPING_DICT)

print('Here are the number of labeled trips remaining in each program dataset:')
expanded_labeled_trips.program.value_counts()

In [ ]:
# Get the confusion matrices and then the EI moments from those.
android_confusion = pd.read_csv("android_confusion.csv").set_index('gt_mode')
ios_confusion = pd.read_csv("ios_confusion.csv").set_index('gt_mode')

android_confusion = cm_handling.collapse_confusion_matrix(android_confusion, rows_to_collapse={"Train": ["Train"]}, columns_to_collapse={})
ios_confusion = cm_handling.collapse_confusion_matrix(ios_confusion, rows_to_collapse={"Train": ["Train"]}, columns_to_collapse={})

# here I'm referring to car_load_factor the number that we divide the drove alone energy intensity by
# for r = 1, car_load_factor is 4/3.
sensed_car_EI = hf.find_sensed_car_energy_intensity(energy_dict, ECAR_PROPORTION, DROVE_ALONE_TO_SHARED_RIDE_RATIO)
energy_dict.update({"Car, sensed": sensed_car_EI})
expanded_labeled_trips['distance_miles'] = expanded_labeled_trips.distance*METERS_TO_MILES
EI_length_cov = 0

In [ ]:
# calculate the energy consumption for each trip before shuffling and splitting.
android_EI_moments_df = cm_handling.get_conditional_EI_expectation_and_variance(android_confusion,energy_dict)
ios_EI_moments_df = cm_handling.get_conditional_EI_expectation_and_variance(ios_confusion,energy_dict)
os_EI_moments_map = {'ios': ios_EI_moments_df, 'android': android_EI_moments_df}
energy_consumption_df = get_EC.compute_all_EC_values(expanded_labeled_trips,unit_dist_MCS_df,energy_dict,android_EI_moments_df,ios_EI_moments_df,
    EI_length_cov, print_info=False)

### Split All CEO into 7 fake programs and then use splits of those programs to separately estimate correlations.
Show that the estimated correlation between percent error and dataset characteristics is roughly the same across fake programs.

#### Run these definition cells
and then the fake programs begin in the following line:  \
fake_programs = get_set_splits(energy_consumption_df,n_rounds=1,n_splits_per_round=7)

In [ ]:
# randomly split CEO dataset many times.
# compute the factors for each split.
# compute the error.
# compute the correlation between each variable in IND_VAR with DEP_VAR
IND_VAR = ['drove_alone_2_shared_ride', 'no_sensed_ratio', 'car_like_ratio', 'e_bike_ratio', 'not_a_trip_ratio',
           "car_like_as_not_car", "e_bike_as_car", "e_bike_as_not_car_bike", 
           "non_car_2_car_user_label", "mispredicted_as_walk", "mispredicted_as_car", 'distance_miles']
DEP_VAR = 'error_pct_for_confusion'

In [ ]:
# There are multiple crossvalidation splitters in sklearn but all of them split into one training and one test set at a time
# If you get this error: "object of type 'int' has no len()", just lower the number of splits per round
def get_set_splits(df, n_rounds = 50, n_splits_per_round=10):
    '''
    Splits data into n_rounds * n_splits_per_round sets.
    n_splits_per_round controls the size of the resulting data subsets. 
    To get lots of datasets without shrinking the size too much, we use multiple rounds of splits.

    Returns numpy array of arrays of data indices.
    '''
    df = df.copy()
    from numpy.random import default_rng
    large_size_splits = []
    for round in range(n_rounds):
        rng = default_rng()
        trip_index = np.array(df.index.copy())
        rng.shuffle(trip_index)
        # print(energy_consumption_df.index, trip_index)

        # splits is a list of numpy arrays of trip indices
        splits = np.array_split(trip_index, n_splits_per_round)

        large_size_splits.append(splits)
    
    unnested_large_size_splits = list(chain.from_iterable(large_size_splits))
    print(f"Subset lengths: {len(unnested_large_size_splits[0])}. Number of subsets: {len(unnested_large_size_splits)}")
    #print([len(s) for s in large_size_splits])

    return unnested_large_size_splits

In [ ]:
#import dataset_splitting_functions as dsf
#dsf.get_splits_and_correlations(energy_consumption_df, IND_VAR, DEP_VAR, n_rounds = 50, n_splits_per_round=10)

In [ ]:
# Several of the split results appear normally distributed about the mean of whatever the dataset used to generate the splits was.
def get_split_results(splits):
    
    CAR_LIKE_MODES = ['drove_alone', 'shared_ride', 'taxi']
    NON_CAR_MOTORIZED_MODES = ['bus', 'free_shuttle', 'train']
    split_result_list = []
    for s in splits:
        ERROR_COLS = ['error_for_confusion',
           'error_for_prediction', 'expected', 'predicted', 'user_labeled', 'distance_miles', 'distance', 'duration']
        curr_split_trips = energy_consumption_df.loc[s]
        curr_split_result = {'count': len(s)}
        for e in ERROR_COLS:
            curr_split_result[e] = curr_split_trips[e].sum()
        curr_split_result['drove_alone_2_shared_ride'] = curr_split_trips.query('mode_confirm == "drove_alone"').distance.sum() / curr_split_trips.query('mode_confirm == "shared_ride"').distance.sum()
        curr_split_result['no_sensed_ratio'] = curr_split_trips.query('primary_mode == "no_sensed"').distance.sum() / curr_split_trips.distance.sum()
        curr_split_result['car_like_ratio'] = curr_split_trips.query('mode_confirm == @CAR_LIKE_MODES').distance.sum() / curr_split_trips.distance.sum()        
        curr_split_result['e_bike_ratio'] = curr_split_trips.query('mode_confirm == "pilot_ebike"').distance.sum() / curr_split_trips.distance.sum()
        curr_split_result['not_a_trip_ratio'] = curr_split_trips.query('mode_confirm == "not_a_trip"').distance.sum() / curr_split_trips.distance.sum()
        
        # car_like_as_not_car: the fraction of car trips that were wrongly labeled as not car. 
        curr_split_result['car_like_as_not_car'] = curr_split_trips.query('mode_confirm == @CAR_LIKE_MODES & primary_mode != "car"').distance.sum() / curr_split_trips.query('mode_confirm == @CAR_LIKE_MODES').distance.sum()
        curr_split_result['e_bike_as_car'] = curr_split_trips.query('mode_confirm == "pilot_ebike" & primary_mode == "car"').distance.sum() / curr_split_trips.query('mode_confirm == "pilot_ebike"').distance.sum()
        curr_split_result['e_bike_as_not_car_bike'] = curr_split_trips.query('mode_confirm == "pilot_ebike" & primary_mode != ["car", "bicycling"]').distance.sum() / curr_split_trips.query('mode_confirm == "pilot_ebike"').distance.sum()

        curr_split_result['non_car_2_car_user_label'] = curr_split_trips.query('mode_confirm == @NON_CAR_MOTORIZED_MODES').distance.sum() / curr_split_trips.query('mode_confirm == @CAR_LIKE_MODES').distance.sum()
        curr_split_result['non_car_2_car_sensed'] = curr_split_trips.query('primary_mode == ["bus", "train"]').distance.sum() / curr_split_trips.query('primary_mode == "car"').distance.sum()
        curr_split_result['mispredicted_as_walk'] = curr_split_trips.query('mode_confirm != "walk" & primary_mode == "walking"').distance.sum() / curr_split_trips.distance.sum()
        curr_split_result['mispredicted_as_car'] = curr_split_trips.query('mode_confirm != @CAR_LIKE_MODES & primary_mode == "car"').distance.sum() / curr_split_trips.distance.sum()
    
        # if curr_split_result['drove_alone_2_shared_ride'] > 0.5:
            # print(f"CHECK: drove_alone %s, shared_ride %s" % (curr_split_trips.query('mode_confirm == "drove_alone"').distance_miles.sum(),
            #                                                   curr_split_trips.query('mode_confirm == "shared_ride"').distance_miles.sum()))
        # print(curr_split_result)
        # print(f"CHECK user_labeled {energy_consumption_df.loc[s].user_labeled.sum()}")
        # print(f"CHECK error_for_confusion {energy_consumption_df.loc[s].error_for_confusion.sum()}")
        split_result_list.append(curr_split_result)
    split_results = pd.DataFrame(split_result_list)
    split_results['error_pct_for_confusion'] = (split_results.error_for_confusion / split_results.user_labeled ) * 100
    split_results['error_pct_for_prediction'] = (split_results.error_for_prediction / split_results.user_labeled) * 100
    return split_results

In [ ]:
def find_correlations(split_results, IND_VAR, DEP_VAR):
    ind_var_correlation_df = pd.DataFrame(columns=["Independent Variable", "Correlation", "p-value"])
    for iv in IND_VAR:
        corr, p = scipy.stats.pearsonr(split_results[iv],split_results[DEP_VAR])
        ind_var_correlation_df = ind_var_correlation_df.append({"Independent Variable": iv, "Correlation": corr, "p-value": p}, ignore_index=True)
    return ind_var_correlation_df.set_index("Independent Variable")

In [ ]:
def get_splits_and_correlations(df,n_rounds = 50, n_splits_per_round=10):
    df = df.copy()
    splits = get_set_splits(df, n_rounds, n_splits_per_round)
    split_results = get_split_results(splits)
    ind_var_correlation_df = find_correlations(split_results, IND_VAR, DEP_VAR)
    return ind_var_correlation_df

In [ ]:
fake_programs = get_set_splits(energy_consumption_df,n_rounds=1,n_splits_per_round=7)

In [ ]:
# Takes approximately 2 min 45 seconds on a Macbook Pro when using all ceo up to May 2022.
fake_program_correlations_map = {}
for i, program in enumerate(fake_programs):
    fake_program_correlations_map[i] = get_splits_and_correlations(energy_consumption_df.loc[program])["Correlation"]

In [ ]:
fake_program_correlations_df = pd.DataFrame(columns=list(range(0,6)))
for i in fake_program_correlations_map.keys():
    fake_program_correlations_df[i] = fake_program_correlations_map[i]
#fake_program_correlations_df["variable"] = IND_VAR

# The factors that were significant when we split the full dataset.
formerly_significant_factors = ["no_sensed_ratio", "car_like_ratio","car_like_as_not_car","e_bike_as_car","non_car_2_car_user_label","mispredicted_as_walk","distance_miles"]
#for up to may 2022: ["car_like_ratio", "e_bike_ratio","car_like_as_not_car", "e_bike_as_car", "mispredicted_as_walk","mispredicted_as_car","distance_miles"]

report_ready_df = fake_program_correlations_df.loc[formerly_significant_factors].copy()
report_ready_df["Independent Variable"] = report_ready_df.index.str.replace('_',' ')
print(report_ready_df.set_index("Independent Variable").round(2).to_latex())

# reset index, round, and latex.

In [ ]:
fake_program_correlations_df.set_index("Independent Variable")

### Try again with larger subsets  of the fake programs.

In [ ]:
for i, program in enumerate(fake_programs):
    fake_program_correlations_map[i] = get_splits_and_correlations(energy_consumption_df.loc[program], n_rounds = 100, n_splits_per_round=5)["Correlation"]

In [ ]:
fake_program_correlations_df = pd.DataFrame(columns=list(range(0,6)))
for i in fake_program_correlations_map.keys():
    fake_program_correlations_df[i] = fake_program_correlations_map[i]
fake_program_correlations_df.round(3)